In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Загрузка архива
uploaded = files.upload()
archive_name = list(uploaded.keys())[0]

# Распаковка архива
with zipfile.ZipFile(archive_name, 'r') as zip_ref:
    zip_ref.extractall('data')

data_dir = 'data'

def load_lattice_data(lattice_dir, label):
    lattice_data = []
    lattice_labels = []
    for lattice_num in os.listdir(lattice_dir):
        lattice_path = os.path.join(lattice_dir, lattice_num)
        if os.path.isdir(lattice_path):
            kde_data = []
            for filename in os.listdir(lattice_path):
                filepath = os.path.join(lattice_path, filename)
                kde_values = pd.read_csv(filepath, header=0).iloc[:, 0].values[1:]  # Пропускаем заголовок
                kde_data.append(kde_values.astype(float))  # Преобразуем в float
            kde_matrix = np.column_stack(kde_data)
            lattice_data.append(kde_matrix)
            lattice_labels.append(label)
    return lattice_data, lattice_labels

# Загрузка данных для каждого типа решётки
lattice_data_7x7x7, labels_7x7x7 = load_lattice_data(os.path.join(data_dir, '7x7x7'), 'NaCl')
lattice_data_UN2, labels_UN2 = load_lattice_data(os.path.join(data_dir, 'UN2'), 'UN2')
lattice_data_U2N3, labels_U2N3 = load_lattice_data(os.path.join(data_dir, 'U2N3'), 'U2N3')

# Объединение данных
X = lattice_data_7x7x7 + lattice_data_UN2 + lattice_data_U2N3
y = np.hstack([np.array(labels_7x7x7), np.array(labels_UN2), np.array(labels_U2N3)])

def pad_matrix(matrix, max_columns):
    padded_matrix = np.pad(matrix, ((0, 0), (0, max_columns - matrix.shape[1])), mode='constant')
    return padded_matrix

# Находим максимальное количество ионов в одной решётке
max_ions = max(matrix.shape[1] for matrix in X)

# Дополняем все матрицы до одинакового размера
X_padded = [pad_matrix(matrix, max_ions) for matrix in X]

# Уплощаем матрицы
def matrix_to_vector(matrix):
    return matrix.flatten()

X_flattened = [matrix_to_vector(matrix) for matrix in X_padded]
X_flattened = np.array(X_flattened)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42, stratify=y)

# Создание и обучение модели
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Подбор гиперпараметров
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")

# Обучение модели с лучшими параметрами
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Accuracy: {accuracy_best:.2f}")

# Использование других моделей
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.2f}")

knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn:.2f}")

# Сохранение модели
joblib.dump(best_model, 'kde_classifier.pkl')


Saving Archive.zip to Archive.zip
Accuracy: 1.00
Лучшие параметры: {'max_depth': None, 'n_estimators': 50}
Best Accuracy: 1.00
SVM Accuracy: 1.00
KNN Accuracy: 1.00


['kde_classifier.pkl']

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib

# Путь к уже распакованной папке с данными
new_data_dir = '/content/new_data'  # или другой путь, где лежат ваши данные

def load_lattice_data(lattice_dir):
    kde_data = []
    for ion_file in os.listdir(lattice_dir):
        filepath = os.path.join(lattice_dir, ion_file)
        if os.path.isfile(filepath) and ion_file.endswith('.csv'):
            try:
                # Чтение CSV файла с заголовком и пропуск первой строки
                kde_values = pd.read_csv(filepath, header=0).iloc[:, 0].values[1:].astype(float)
                kde_data.append(kde_values)
                print(f"Успешно загружен файл: {ion_file}, размер: {len(kde_values)}")
            except Exception as e:
                print(f"Ошибка при чтении файла {filepath}: {e}")

    if not kde_data:
        raise ValueError(f"Не найдено ни одного корректного файла в папке {lattice_dir}")

    kde_matrix = np.column_stack(kde_data)
    print(f"Создана матрица размером: {kde_matrix.shape}")
    return kde_matrix

# Путь к конкретной папке с решеткой
lattice_dir = os.path.join(new_data_dir, '21')

# Проверка существования папки
if not os.path.exists(lattice_dir):
    print(f"Папка {lattice_dir} не существует!")
    print("Доступные папки в new_data:")
    for item in os.listdir(new_data_dir):
        item_path = os.path.join(new_data_dir, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}")
            # Показать подпапки
            for subitem in os.listdir(item_path):
                subitem_path = os.path.join(item_path, subitem)
                if os.path.isdir(subitem_path):
                    print(f"    📂 {subitem}")
else:
    print(f"Путь к папке с данными: {lattice_dir}")
    print(f"Список файлов в папке: {os.listdir(lattice_dir)[:10]}...")  # Показываем первые 10 файлов

    # Загрузка данных для новой решётки
    try:
        kde_matrix = load_lattice_data(lattice_dir)
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        raise

    # Загрузка сохранённой модели
    model = joblib.load('kde_classifier.pkl')

    def pad_matrix(matrix, max_columns):
        padded_matrix = np.pad(matrix, ((0, 0), (0, max_columns - matrix.shape[1])), mode='constant')
        return padded_matrix

    # Максимальное количество ионов в одной решётке из обучающей выборки
    max_ions = 343  # Замените на актуальное значение из вашей обученной модели

    # Дополняем матрицу до одинакового размера
    kde_matrix_padded = pad_matrix(kde_matrix, max_ions)
    print(f"Матрица после дополнения: {kde_matrix_padded.shape}")

    # Уплощаем матрицу
    def matrix_to_vector(matrix):
        return matrix.flatten()

    kde_vector = matrix_to_vector(kde_matrix_padded)
    kde_vector = kde_vector.reshape(1, -1)  # Преобразуем в двумерный массив (одна строка)
    print(f"Вектор для классификации: {kde_vector.shape}")

    # Предсказание типа решётки
    predicted_label = model.predict(kde_vector)
    prediction_proba = model.predict_proba(kde_vector)

    print(f"Предсказанный тип решетки: {predicted_label[0]}")
    print(f"Вероятности:")

    for i, (class_name, prob) in enumerate(zip(model.classes_, prediction_proba[0])):
        print(f"  {class_name}: {prob:.4f} ({prob*100:.2f}%)")

Путь к папке с данными: /content/new_data/21
Список файлов в папке: ['kde_array_777_Cl_0.84807903_0.15876380_0.65043946.csv', 'kde_array_777_Na_0.47827458_0.00000000_0.33950642.csv', 'kde_array_777_Na_0.69308471_0.98859405_0.15734638.csv', 'kde_array_777_Na_0.49747574_0.83338360_0.50582287.csv', 'kde_array_777_Cl_0.78789024_0.33203705_0.82587566.csv', 'kde_array_777_Na_0.16351575_0.84692486_0.49599104.csv', 'kde_array_777_Na_0.33624272_0.47048582_0.31558941.csv', 'kde_array_777_Cl_0.83805043_0.88304947_1.02850603.csv', 'kde_array_777_Na_0.17387392_0.85082347_0.16254473.csv', 'kde_array_777_Cl_0.32857575_0.34173389_0.00000000.csv']...
Успешно загружен файл: kde_array_777_Cl_0.84807903_0.15876380_0.65043946.csv, размер: 999
Успешно загружен файл: kde_array_777_Na_0.47827458_0.00000000_0.33950642.csv, размер: 999
Успешно загружен файл: kde_array_777_Na_0.69308471_0.98859405_0.15734638.csv, размер: 999
Успешно загружен файл: kde_array_777_Na_0.49747574_0.83338360_0.50582287.csv, размер: 99